# Trabalho ENGA75

### Autor: Allan Souza Almeida

#### Importando objeto UR5 e suas funções

In [ ]:
from ur5 import UR5, forward_kinematics
import math
import numpy as np
import matplotlib.pyplot as plt
import time
PI = math.pi
ur5 = UR5()

## Etapa A1

Implementar a Cinemática Direta e Inversa do Robô UR5. Validar a modelagem com o auxílio do
simulador para um conjunto de valores das variáveis das juntas (Cinemática Direta) e para um conjunto de
posições e orientações da garra (Cinemática Inversa)

#### Cinemática Direta

<img src="Cinemática Direta (DH).png" width="500">

In [ ]:
def validate_fk(ang):
    fk, _ = forward_kinematics(ang)    # Calcular cinemática direta a partir dos ângulos
    print('Cinemática Direta: ')
    print(fk)
    ur5.move_to_config(ang)     # Mover para a posição
    print('Ground Truth: ')
    gt = ur5.get_ground_truth()     # Obter Ground Truth
    print(gt)
    print('Erro: ', np.linalg.norm(fk-gt)/np.linalg.norm(gt)*100, '%')  # Calcular e printar erro

In [ ]:
validate_fk([.5*PI, .05*PI, .45*PI, 0, -.5*PI, .5*PI])

In [ ]:
validate_fk([-.5*PI, -.05*PI, -.45*PI, 0, .5*PI, .5*PI])

In [ ]:
validate_fk([PI/2, 0, 0, 0, 0, 0])

In [ ]:
validate_fk([.1*PI, .18*PI, .57*PI, 0.25*PI, 0.2, PI/2])

In [ ]:
validate_fk([-.3*PI, -.04*PI, -.8*PI, -0.17*PI, -0.5, PI/2])

In [ ]:
validate_fk([0.2, .04*PI, .8*PI, 0.17*PI, 0.3, PI/2])

#### Cinemática Inversa

<img src="gripper.png" width="700">

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [PI, 0, PI/2], wrist='up')

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [0, 0, -PI/2], wrist='down')

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [PI/2, 0, -PI/2], wrist='down')

In [ ]:
ur5.move_to_pose([-0.1, -.45, .4], [PI/2, 0, -PI/2], wrist='down')

In [ ]:
ur5.move_to_pose([-0.1, -.45, .25], [PI/2, 0, -PI/2], wrist='down')

In [ ]:
ur5.move_to_pose([-0.1, -.45, .25], [0, -PI/2, PI], wrist='up')

## Etapa A2

Implementar um sistema de planejamento de trajetórias no espaço de juntas com uma das abordagens
abaixo:

1. Trajetórias polinomiais cúbicas ou de quinto grau;
2. Linear Segments with Parabolic Blends (LSPB);
3. Minimum Time Trajectories

<b>Polinômio de Quinto Grau</b>

<p>$q(t) = a_0+a_1 t + a_2 t^2 + a_3 t^3 + a_4 t^4 + a_5 t^5$</p>

<p>$\dot{q}(t) = a_1 + 2 a_2 t + 3 a_3 t^2 + 4 a_4 t^3 + 5 a_5 t^4$</p>

<p>$\ddot{q}(t) = 2 a_2 + 6 a_3 t + 12 a_4 t^2 + 20 a_5 t^3$</p>

$\begin{bmatrix}
1 & t_0 & t_0^2 & t_0^3 & t_0^4 & t_0^5\\
0 & 1 & 2t_0 & 3t_0^2 & 4t_0^3 & 5t_0^4\\
0 & 0 & 2 & 6t_0 & 12t_0^2 & 20t_0^3\\
1 & t_f & t_f^2 & t_f^3 & t_f^4 & t_f^5\\
0 & 1 & 2t_f & 3t_f^2 & 4t_f^3 & 5t_f^4\\
0 & 0 & 2 & 6t_f & 12t_f^2 & 20t_f^3\\
\end{bmatrix}

\cdot

\begin{bmatrix}
a_0\\
a_1\\
a_2\\
a_3\\
a_4\\
a_5\\
\end{bmatrix}

=

\begin{bmatrix}
q_0\\
\dot{q}_0\\
\ddot{q}_0\\
q_f\\
\dot{q}_f\\
\ddot{q}_f\\
\end{bmatrix}$

<p>$A\cdot X = B$</p>

In [ ]:
t, max_err, mean_err, graphs, _ = ur5.move_to_config([.5*PI, .05*PI, .45*PI, 0, .5*PI, .3*PI], graph=True)

In [ ]:
titles = ['Posição', 'Velocidade', 'Aceleração', 'Solavanco']
axes = ['rad', 'rad/s', 'rad/$s^2$', 'rad/$s^3$']

for i in range(4):
    for j in range(6):
        plt.plot(graphs[4][j], graphs[i][j])
    plt.legend(['q1', 'q2', 'q3', 'q4', 'q5', 'q6'])
    plt.grid()
    plt.title(titles[i], fontsize=18)
    plt.ylabel(axes[i], rotation=0, labelpad=20)
    plt.xlabel('t [s]')
    plt.show()

## Jacobiano

In [ ]:
_, _, _, _, _ = ur5.move_to_config([0, 0, 0, 0, 0, 0])

In [ ]:
_, _, _, graphs, jacob = ur5.move_to_config([0, 0, -PI/2, 0, 0, 0], graph=True, jacob=True, duration=None)

In [ ]:
for j in range(6):
    plt.plot(graphs[4][j], graphs[1][j])
plt.legend(['q1', 'q2', 'q3', 'q4', 'q5', 'q6'])
plt.grid()
plt.title('Velocidade', fontsize=18)
plt.ylabel('rad/s', rotation=0, labelpad=20)
plt.xlabel('t [s]')
plt.show()

In [ ]:
for j in range(3):
    plt.plot(graphs[4][j], [i[j] for i in jacob])
plt.title('Jacobiano (vel. lin.)', fontsize=18)
plt.legend(['$v_x$', '$v_y$', '$v_z$'])
plt.xlabel('t [s]')
plt.ylabel('m/s', rotation=0, labelpad=20)
plt.show()

for j in range(3, 6):
    plt.plot(graphs[4][j], [i[j] for i in jacob])
plt.title('Jacobiano (vel,. ang.)', fontsize=18)
plt.legend(['$\omega_x$', '$\omega_y$', '$\omega_z$'])
plt.xlabel('t [s]')
plt.ylabel('rad/s', rotation=0, labelpad=20)
plt.show()

## Visão Computacional e Pick and Place

<img src="vgg16.jpg">

#### Pose de Leitura de Imagem

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [PI, 0, PI/2], wrist='up')

#### VGG16 - Transfer Learning - Visão Computacional

In [ ]:
cup_position = ur5.move_cup()
predicted_cup_position = ur5.predict()
print('Posição real do copo: ', cup_position)
print('Posição predita do copo: ', predicted_cup_position)
print('Erro: ', np.linalg.norm(cup_position - predicted_cup_position)*100, 'cm')

#### Pre-Graspping

In [ ]:
ur5.move_to_pose(predicted_cup_position+np.array([0, 0.2, 0.015]), [PI/2, 0, -PI/2])

#### Graspping

In [ ]:
ur5.move_to_pose(predicted_cup_position + np.array([0, 0.018, 0.017]), [PI / 2, 0, -PI / 2], duration=6)

In [ ]:
ur5.close_gripper()

In [ ]:
ur5.move_to_pose(predicted_cup_position + np.array([0, 0.018, 0.047]), [PI / 2, 0, -PI / 2])

#### Pre-Placing

In [ ]:
ur5.move_to_pose(np.array([0.4, 0, 0.09]), [PI/2, 0, -PI/2], wrist='up', duration=10)

#### Placing

In [ ]:
ur5.move_to_pose(np.array([0.4, 0, 0.06]), [PI/2, 0, -PI/2], wrist='up')

In [ ]:
ur5.open_gripper()
time.sleep(2)

#### Afastando do copo

In [ ]:
ur5.move_to_pose(np.array([0.4, .1, predicted_cup_position[2]+0.01]), [PI/2, 0, -PI/2], wrist='up')

In [ ]:
ur5.move_to_pose(np.array([0.4, .1, predicted_cup_position[2]+0.2]), [PI/2, 0, -PI/2], wrist='up')

#### Voltando para a posição inicial

In [ ]:
ur5.move_to_pose([-0.1, -.55, .4], [PI, 0, PI/2], wrist='up')